<a href="https://colab.research.google.com/github/mak109/cs6910_assignment2/blob/main/PART%20A/cs6910_assignment2_partA_question5_guided_backprop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Guided Backpropagation Implementation
In  question 5 we implemented the following things
- Guided Backprop on CONV5 layer
- Activations visualized for ten neurons(or feature maps) for CONV5 layer



In [ ]:
!pip install wget

In [ ]:
!pip install wandb -qqq
import wandb
wandb.login()

wandb: Currently logged in as: mak109 (use `wandb login --relogin` to force relogin)


True

In [ ]:
from wandb.keras import WandbCallback

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
import numpy as np
import random
import wget
import os
from zipfile import ZipFile
from PIL import Image
from matplotlib import gridspec
plt.rcParams["figure.figsize"] = (20,10)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers,Sequential,regularizers,optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import random
random.seed(123)

In [ ]:
import wget
import os
from zipfile import ZipFile
url='https://storage.googleapis.com/wandb_datasets/nature_12K.zip'
filename = os.path.basename(url)

if not os.path.exists(filename) and not os.path.exists("inaturalist_12K"):
  filename = wget.download(url)
  with ZipFile(filename, 'r') as zip:
    print('Extracting all the files now...')
    zip.extractall()
    print('Done!')
  os.remove(filename)

In [ ]:
image_size = (256,256)
num_layers = 5
num_classes = 10

In [ ]:
# tf.debugging.set_log_device_placement(True)

In [ ]:
def CNN(config):
    model = Sequential([
        layers.Input((image_size[0],image_size[1],3)),
        layers.Rescaling(1./255)
        ])
    
    for l in range(num_layers):
        model.add(layers.Conv2D(filters=config["filters_list"][l],kernel_size=(config["kernel_sizes"][l][0],config["kernel_sizes"][l][1]),
                        activation=config["activation"],padding="same",kernel_regularizer=regularizers.l2(config["weight_decay"])))
        if config["batch_normalization"] == 'True':
            model.add(layers.BatchNormalization())
        model.add(layers.MaxPooling2D(pool_size=(2,2)))

    model.add(layers.Flatten())
    model.add(layers.Dense(config["dense_layer_size"],activation=config["activation"],kernel_regularizer=regularizers.l2(config["weight_decay"])))
    model.add(layers.Dropout(config["dropout"]))

    model.add(layers.Dense(num_classes,activation="softmax"))
    return model



In [ ]:
# best_test_run_id = "8ohaf75c"
test_run_id = "vyo6dtkg"
api = wandb.Api()
run = api.run("dlstack/CS6910-ASSIGNMENT-2/"+test_run_id)
if os.path.exists('model-best.h5'):
    os.remove('model-best.h5')
model_weights = wandb.restore("model-best.h5",run_path="dlstack/CS6910-ASSIGNMENT-2/"+test_run_id)
config = run.config

model = CNN(config)
model.load_weights(model_weights.name)
test_generator = ImageDataGenerator(dtype=tf.float32,validation_split=0.0,data_format='channels_last').flow_from_directory(
        'inaturalist_12K/val',
        target_size = image_size,
        batch_size = config['batch_size'],
        color_mode = 'rgb',
        class_mode = 'sparse',
        seed=123
        )
test_generator_ = test_generator
images,labels = next(iter(test_generator_))
# model.compile(
#     optimizer=optimizers.Adam(learning_rate=config["learning_rate"]),
#     loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
#     metrics=['accuracy']
#     )

Found 2000 images belonging to 10 classes.


In [ ]:
class_names =list(test_generator.class_indices)

In [ ]:
@tf.custom_gradient
def guidedRelu(x):
    def grad(dy):
        return tf.cast(dy>0,"float32") * tf.cast(x>0, "float32") * dy
    return tf.nn.relu(x), grad

In [ ]:
layer_activation_list = [layer.activation for layer in model.layers if hasattr(layer,'activation')]
activation_model = tf.keras.models.Model([model.inputs],[model.layers[9].output])
for activation in layer_activation_list:
    if activation == tf.keras.activations.relu:
        activation = guidedRelu
        

In [ ]:
with tf.GradientTape() as tape:
    inputs = tf.cast(images,tf.float32)
    tape.watch(inputs)
    outputs = activation_model(inputs)
grads = tape.gradient(outputs,inputs)

In [ ]:
wandb.init(project="CS6910-ASSIGNMENT-2",entity="dlstack")
plt.figure(figsize=(200,200))
plt.title("Activations of CONV5 layer")
g = gridspec.GridSpec(12,10,hspace=0.1,wspace=0.1,left=0.8,right=0.9,top=0.9,bottom=0.8)
for j in range(10):
    ax = plt.subplot(g[0,j])
    ax.imshow(images[j].astype("uint8"))
    ax.set_title(class_names[labels[j].astype("int")])
    ax.axis("off")

    for i in range(10):
        ax = plt.subplot(g[i+1,j])
        ax.imshow(outputs.numpy()[j,:,:,i],cmap='gray')
        ax.axis("off")
    gb_viz = grads[j]
    gb_viz = np.dstack((
                gb_viz[:, :, 0],
                gb_viz[:, :, 1],
                gb_viz[:, :, 2],
            ))       
    gb_viz -= np.min(gb_viz)
    gb_viz /= gb_viz.max()

    ax = plt.subplot(g[11,j])
    ax.imshow(gb_viz)
    ax.axis("off")
plt.savefig("guided_backprop.jpg",bbox_inches="tight")
wandb.run.name= "guided-backprop"
wandb.log({"Guided backprop on 10 images for CONV5 layer":wandb.Image("guided_backprop.jpg")})
wandb.finish()